# Control Cinemático 4 GDL espacial

Seguimiento de una trayectoria circular en el plano XZ para un manipulador con cuatro articulaciones rotacionales (4 eslabones) controlando posición $(x, y, z)$ y orientación de pitch.

## 1. Configuración de simulación

In [ ]:
import sys
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

plt.rcParams['animation.embed_limit'] = 80

ROOT = pathlib.Path().resolve().parents[1]
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from notebooks.control.utils import time_vector

In [ ]:
sim_config = {"t0": 0.0, "tf": 24.0, "n_steps": 720}
radius = 0.12
center_x = 0.25
center_z = 0.25
constant_y = 0.04
omega = 2 * np.pi / (sim_config["tf"] - sim_config["t0"])

t, dt = time_vector(sim_config)

x_ref = center_x + radius * np.cos(omega * t)
y_ref = np.full_like(t, constant_y)
z_ref = center_z + radius * np.sin(omega * t)

xd_dot = np.gradient(x_ref, dt)
yd_dot = np.gradient(y_ref, dt)
zd_dot = np.gradient(z_ref, dt)

pitch_ref = np.unwrap(np.arctan2(zd_dot, xd_dot))
pitch_dot = np.gradient(pitch_ref, dt)

Xd = np.vstack((x_ref, y_ref, z_ref, pitch_ref))
Vd = np.vstack((xd_dot, yd_dot, zd_dot, pitch_dot))

L1 = 0.12
L2 = 0.18
L3 = 0.16
L4 = 0.12

## 2. Modelo cinemático
Se definen las funciones de cinemática directa y Jacobiano analítico para el vector de estado $[x, y, z, 	heta]$ (pitch).

In [ ]:
def forward_kinematics(q: np.ndarray) -> np.ndarray:
    q1, q2, q3, q4 = q
    c1, s1 = np.cos(q1), np.sin(q1)
    s2, c2 = np.sin(q2), np.cos(q2)
    s23, c23 = np.sin(q2 + q3), np.cos(q2 + q3)
    s234, c234 = np.sin(q2 + q3 + q4), np.cos(q2 + q3 + q4)

    radial = L2 * c2 + L3 * c23 + L4 * c234
    x = c1 * radial
    y = s1 * radial
    z = L1 + L2 * s2 + L3 * s23 + L4 * s234
    pitch = q2 + q3 + q4
    return np.array([x, y, z, pitch])


def jacobian(q: np.ndarray) -> np.ndarray:
    q1, q2, q3, q4 = q
    c1, s1 = np.cos(q1), np.sin(q1)
    s2, c2 = np.sin(q2), np.cos(q2)
    s23, c23 = np.sin(q2 + q3), np.cos(q2 + q3)
    s234, c234 = np.sin(q2 + q3 + q4), np.cos(q2 + q3 + q4)

    radial = L2 * c2 + L3 * c23 + L4 * c234
    dr_dq2 = -L2 * s2 - L3 * s23 - L4 * s234
    dr_dq3 = -L3 * s23 - L4 * s234
    dr_dq4 = -L4 * s234

    dz_dq2 = L2 * c2 + L3 * c23 + L4 * c234
    dz_dq3 = L3 * c23 + L4 * c234
    dz_dq4 = L4 * c234

    J = np.array(
        [
            [-s1 * radial, c1 * dr_dq2, c1 * dr_dq3, c1 * dr_dq4],
            [c1 * radial, s1 * dr_dq2, s1 * dr_dq3, s1 * dr_dq4],
            [0.0, dz_dq2, dz_dq3, dz_dq4],
            [0.0, 1.0, 1.0, 1.0],
        ]
    )
    return J

## 3. Control cinemático

In [ ]:
Kp = np.diag([8.0, 8.0, 8.0, 4.0])
q = np.zeros((4, t.size))
q[:, 0] = np.array([0.0, np.pi / 6, -np.pi / 6, 0.0])
X = np.zeros((4, t.size))

for k in range(t.size - 1):
    X[:, k] = forward_kinematics(q[:, k])
    error = Xd[:, k] - X[:, k]
    error[3] = (error[3] + np.pi) % (2 * np.pi) - np.pi
    v_ref = Vd[:, k] + Kp @ error
    J = jacobian(q[:, k])
    dq = np.linalg.pinv(J) @ v_ref
    q[:, k + 1] = q[:, k] + dq * dt

X[:, -1] = forward_kinematics(q[:, -1])

## 4. Validación del error

In [ ]:
pos_error = np.linalg.norm(Xd[:3] - X[:3], axis=0)
pitch_error = ((Xd[3] - X[3]) + np.pi) % (2 * np.pi) - np.pi
max_pos_error = float(pos_error.max())
max_pitch_error = float(np.abs(pitch_error).max())
print(f"Error máximo de posición: {max_pos_error:.4f} m")
print(f"Error máximo de pitch: {np.degrees(max_pitch_error):.2f} grados")
assert max_pos_error < 0.03, "El error de posición excede 3 cm"
assert max_pitch_error < np.deg2rad(3.0), "El error de pitch excede 3 grados"

## 5. Gráficas de seguimiento

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
labels = ["x", "y", "z", "pitch"]
for i, ax in enumerate(axes.flat):
    ax.plot(t, Xd[i], "--", label="Deseada")
    ax.plot(t, X[i], label="Seguimiento")
    ax.set_xlabel("Tiempo [s]")
    ax.set_ylabel(labels[i])
    ax.legend()
    ax.grid(True)
fig.tight_layout()
fig

## 6. Animación 3D

In [ ]:
def arm_points(q: np.ndarray) -> np.ndarray:
    q1, q2, q3, q4 = q
    c1, s1 = np.cos(q1), np.sin(q1)
    s2, c2 = np.sin(q2), np.cos(q2)
    s23, c23 = np.sin(q2 + q3), np.cos(q2 + q3)
    s234, c234 = np.sin(q2 + q3 + q4), np.cos(q2 + q3 + q4)

    base = np.array([0.0, 0.0, 0.0])
    joint1 = np.array([0.0, 0.0, L1])

    r2 = L2 * c2
    z2 = L1 + L2 * s2
    joint2 = np.array([c1 * r2, s1 * r2, z2])

    r3 = L2 * c2 + L3 * c23
    z3 = L1 + L2 * s2 + L3 * s23
    joint3 = np.array([c1 * r3, s1 * r3, z3])

    r4 = L2 * c2 + L3 * c23 + L4 * c234
    z4 = L1 + L2 * s2 + L3 * s23 + L4 * s234
    ee = np.array([c1 * r4, s1 * r4, z4])

    return np.stack((base, joint1, joint2, joint3, ee), axis=1)


fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection="3d")
ax.set_xlim(0.0, 0.6)
ax.set_ylim(-0.4, 0.4)
ax.set_zlim(0.0, 0.6)
ax.set_xlabel("x [m]")
ax.set_ylabel("y [m]")
ax.set_zlabel("z [m]")
ax.set_title("Trayectoria circular en plano XZ")
ax.plot(x_ref, y_ref, z_ref, "--", color="tab:orange", label="Trayectoria deseada")
arm_line, = ax.plot([], [], [], "o-", color="tab:blue", lw=2, label="Robot")
ax.legend(loc="upper left")

frames = range(0, t.size, 3)


def init():
    arm_line.set_data([], [])
    arm_line.set_3d_properties([])
    return (arm_line,)


def animate(i):
    pts = arm_points(q[:, i])
    arm_line.set_data(pts[0], pts[1])
    arm_line.set_3d_properties(pts[2])
    return (arm_line,)


anim = animation.FuncAnimation(
    fig, animate, init_func=init, frames=frames, interval=40, blit=True
)
HTML(anim.to_jshtml())